# Reshape o pivotear un Dataframe en Pandas

https://pandas.pydata.org/docs/user_guide/reshaping.html
https://sites.ualberta.ca/~hadavand/DataAnalysis/notebooks/Reshaping_Pandas.html
https://ai.plainenglish.io/reshaping-and-pivoting-in-pandas-a41678e72d68
https://stackoverflow.com/questions/71658101/how-to-reshape-a-pivot-table


https://analyticsindiamag.com/how-to-create-a-pivot-table-in-python-from-scratch/#:~:text=A%20Pivot%20table%20is%20an,variables%20based%20on%20categorical%20variables.
https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
https://www.machinelearningplus.com/pandas/pandas-pivot-table-in-python/
https://datagy.io/python-pivot-tables/    ---> very cool 
https://www.analyticsvidhya.com/blog/2020/03/pivot-table-pandas-python/
https://www.youtube.com/watch?v=hA4l2ePFjfM&ab_channel=FrankAndrade
https://www.youtube.com/watch?v=OJDBUXNySWI&ab_channel=ShwetaLodha
https://builtin.com/data-science/pandas-pivot-tables



 To create a spreadsheet-style pivot table as a data frame in python, we use pandas.pivot_table() function. 

 pandas.pivot_table(data, 
                    values=None, 
                      index=None, 
                      columns=None, 
                      aggfunc='mean',
                      fill_value=None, 
                       margins=False, 
                       dropna=True, 
                       margins_name='All', 
                       observed=False,
                         sort=True)

data: data frame ( Defining the dataset that is to be used for the pivot table.) 
values: column to aggregate ( Feature that is to be seen in its statistical summary.)
index: column ( Indexes the value passed in the value argument )
columns: column ( For aggregating values based on certain features )
aggfunc: function or list of functions ( Aggregating functions like sum, mean, etc )
fill_value: scalar ( Value to replace missing values in the table )
margins: bool ( Add all row / columns ( e.g. for subtotal / grand totals ) )

## 1. Reshaping by pivoting DataFrame objects
## 2. Reshaping by melt
## . Pivot tables
## 4. Examples 


## 1. Reshaping by pivoting DataFrame objects

In [4]:
import numpy as np
import pandas as pd

import pandas._testing as tm
def unpivot(frame):
    N, K = frame.shape
    data = {
        "value": frame.to_numpy().ravel("F"),
        "variable": np.asarray(frame.columns).repeat(N),
        "date": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["date", "variable", "value"])
df = unpivot(tm.makeTimeDataFrame(3))
df

,date,variable,value
0,2000-01-03,A,-0.680165
1,2000-01-04,A,0.895744
2,2000-01-05,A,1.172734
3,2000-01-03,B,-1.302627
4,2000-01-04,B,1.159100
5,2000-01-05,B,-0.031234
6,2000-01-03,C,-2.710837
7,2000-01-04,C,-0.606683
8,2000-01-05,C,1.550922
9,2000-01-03,D,-0.353418


In [5]:
pivoted = df.pivot(index="date", columns="variable", values="value")
pivoted

variable,A,B,C,D
date,,,,
2000-01-03,-0.680165,-1.302627,-2.710837,-0.353418
2000-01-04,0.895744,1.159100,-0.606683,1.620844
2000-01-05,1.172734,-0.031234,1.550922,-0.423936


In [6]:
df["value2"] = df["value"] * 2

pivoted_2 = df.pivot(index="date", columns="variable")
pivoted_2

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -0.680165 -1.302627 -2.710837 -0.353418 -1.360330 -2.605254   
2000-01-04  0.895744  1.159100 -0.606683  1.620844  1.791488  2.318200   
2000-01-05  1.172734 -0.031234  1.550922 -0.423936  2.345468 -0.062469   

                                
variable           C         D  
date                            
2000-01-03 -5.421675 -0.706836  
2000-01-04 -1.213366  3.241687  
2000-01-05  3.101844 -0.847872

## 2. Reshaping by stacking and unstacking

In [8]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)

index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df2 = df[:4]
df2

A         B
first second                    
bar   one    -0.161233  0.499626
      two    -0.870197  0.101622
baz   one     0.745269  0.055302
      two    -1.688234  2.273350

In [10]:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -0.161233
               B    0.499626
       two     A   -0.870197
               B    0.101622
baz    one     A    0.745269
               B    0.055302
       two     A   -1.688234
               B    2.273350
dtype: float64

In [11]:
stacked.unstack()

A         B
first second                    
bar   one    -0.161233  0.499626
      two    -0.870197  0.101622
baz   one     0.745269  0.055302
      two    -1.688234  2.273350

In [12]:
stacked.unstack("second")

second        one       two
first                      
bar   A -0.161233 -0.870197
      B  0.499626  0.101622
baz   A  0.745269 -1.688234
      B  0.055302  2.273350

In [13]:
index = pd.MultiIndex.from_product([[2, 1], ["a", "b"]])
df = pd.DataFrame(np.random.randn(4), index=index, columns=["A"])
df

A
2 a -0.370824
  b -1.333540
1 a  0.244229
  b -0.406280

## 3. Reshaping by melt

In [15]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)

cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [18]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [19]:
cheese.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [20]:
index = pd.MultiIndex.from_tuples([("person", "A"), ("person", "B")])

cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    },
    index=index,
)

cheese

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [21]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [22]:
cheese.melt(id_vars=["first", "last"], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

Another way to transform is to use the wide_to_long() panel data convenience function. It is less flexible than melt(), but more user-friendly.

In [23]:
dft = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
    }
)

dft["id"] = dft.index

dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-0.302934,0
1,b,e,1.2,1.3,0.785428,1
2,c,f,0.7,0.1,-0.825823,2


In [24]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-0.302934,a,2.5
1,1970,0.785428,b,1.2
2,1970,-0.825823,c,0.7
0,1980,-0.302934,d,3.2
1,1980,0.785428,e,1.3
2,1980,-0.825823,f,0.1


## 5. Pivot tables

While pivot() provides general purpose pivoting with various data types (strings, numerics, etc.), pandas also provides pivot_table() for pivoting with aggregation of numeric data.

The function pivot_table() can be used to create spreadsheet-style pivot tables. See the cookbook for some advanced strategies.

It takes a number of arguments:

data: a DataFrame object.

values: a column or a list of columns to aggregate.

index: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

columns: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

aggfunc: function to use for aggregation, defaulting to numpy.mean.

In [26]:
import datetime

In [28]:
df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)]
        + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
    }
)

df

,A,B,C,D,E,F
0,one,A,foo,-0.118853,-0.320646,2013-01-01
1,one,B,foo,-1.078602,0.166013,2013-02-01
2,two,C,foo,-0.420980,1.285236,2013-03-01
3,three,A,bar,-1.991333,-0.543341,2013-04-01
4,one,B,bar,-1.497076,0.459684,2013-05-01
5,one,C,bar,-0.793647,-0.647399,2013-06-01
6,two,A,foo,0.310138,-1.727177,2013-07-01
7,three,B,foo,0.192958,0.407213,2013-08-01
8,one,C,foo,-1.355529,0.062304,2013-09-01
9,one,A,bar,0.481101,-0.128802,2013-10-01


In [29]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

C             bar       foo
A     B                    
one   A -0.281934 -0.505055
      B -0.501710  0.191514
      C -0.365315 -0.376109
three A -1.524297       NaN
      B       NaN  0.617305
      C  1.011832       NaN
two   A       NaN  0.231923
      B -0.601567       NaN
      C       NaN  0.665402

In [32]:
pd.pivot_table(df, values="D", index=["B"], columns=["A", "C"], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A -0.563869 -1.010111 -3.048594       NaN       NaN  0.463847
B -1.003419  0.383027       NaN  1.234609 -1.203134       NaN
C -0.730630 -0.752218  2.023664       NaN       NaN  1.330804

In [35]:
pd.pivot_table(
    df, 
   # values=["D", "E"],
    index=["B"],
    columns=["A", "C"],
    aggfunc=np.sum,
)

C:\Users\oscar\AppData\Local\Temp\ipykernel_2412\3326200049.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pd.pivot_table(


D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A -0.563869 -1.010111 -3.048594       NaN       NaN  0.463847 -2.360373   
B -1.003419  0.383027       NaN  1.234609 -1.203134       NaN  0.500975   
C -0.730630 -0.752218  2.023664       NaN       NaN  1.330804  0.332256   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A  0.457805 -0.138202       NaN       NaN -1.177650  
B -0.884732       NaN -0.433356  0.990779       NaN  
C -1.834528  0.456906       NaN       NaN  3.818924

## 6. Examples 

In [36]:
import pandas as pd
import numpy as np

# data

df = pd.DataFrame({'Name': ['Minecraft', 'Grand Theft Auto V', 'Tetris (EA)', 'Wii Sports', 'PUBG: Battlegrounds', ],
                   'Genre': ['Survival,Sandbox', 'Action-adventure', 'Puzzle', 'Sports simulation', 'Battle royale'],
                   'Platform': ['Multi-platform', 'Multi-platform', 'Multi-platform', 
                            'Wii', 'PC'],
                   'Publishers': ['Xbox Game Studios', 'Rockstar Games', 'Electronic Arts', 
                                  'Nintendo', 'PUBG Corporation'],
                   'Total_Year': [9, 7, 14, 10, 5],
                   'Sales': [238, 160, 100, 82, 75]})
df

,Name,Genre,Platform,Publishers,Total_Year,Sales
0,Minecraft,"Survival,Sandbox",Multi-platform,Xbox Game Studios,9,238
1,Grand Theft Auto V,Action-adventure,Multi-platform,Rockstar Games,7,160
2,Tetris (EA),Puzzle,Multi-platform,Electronic Arts,14,100
3,Wii Sports,Sports simulation,Wii,Nintendo,10,82
4,PUBG: Battlegrounds,Battle royale,PC,PUBG Corporation,5,75


In [40]:
# Create pivot table using pandas

table = pd.pivot_table( data=df, 
                        index=['Platform'], 
                        columns=['Publishers'], 
                        values='Sales',
                        aggfunc='mean')
table


Publishers,Electronic Arts,Nintendo,PUBG Corporation,Rockstar Games,Xbox Game Studios
Platform,,,,,
Multi-platform,100.0,NaN,NaN,160.0,238.0
PC,NaN,NaN,75.0,NaN,NaN
Wii,NaN,82.0,NaN,NaN,NaN


In [41]:
# Using multiple aggregation functions

# if the column parameter is not specified, it will aggregate based on the index. So, let’s not specify the column parameter and see what changes happen in our pivot table.

table2 = pd.pivot_table(data=df, 
                        index=['Platform'],
                        values='Sales',
                        aggfunc=['sum', 'mean', 'count'])
table2



,sum,mean,count
,Sales,Sales,Sales
Platform,,,
Multi-platform,498,166,3
PC,75,75,1
Wii,82,82,1


In [42]:
# ! As discussed above, if the column parameter is not specified, the function itself aggregates on the index parameter.


# Aggregating for multiple features and specific features

## Additionally, we can perform different aggregations based on different features. The result is that multiple pivot tables need not be created to apply appropriate operations to different features.

table3 = pd.pivot_table(data=df, 
                        index='Platform', 
                        values=['Sales', 'Total_Year'],
                        columns=['Publishers'],
                        aggfunc={'Sales': np.sum, 'Total_Year': np.mean})
table3



Sales                                           \
Publishers     Electronic Arts Nintendo PUBG Corporation Rockstar Games   
Platform                                                                  
Multi-platform           100.0      NaN              NaN          160.0   
PC                         NaN      NaN             75.0            NaN   
Wii                        NaN     82.0              NaN            NaN   

                                      Total_Year                            \
Publishers     Xbox Game Studios Electronic Arts Nintendo PUBG Corporation   
Platform                                                                     
Multi-platform             238.0            14.0      NaN              NaN   
PC                           NaN             NaN      NaN              5.0   
Wii                          NaN             NaN     10.0              NaN   

                                                 
Publishers     Rockstar Games Xbox Game Studios  
Platform                                         
Multi-platform            7.0               9.0  
PC                        NaN               NaN  
Wii                       NaN               NaN

In [43]:
# Replacing missing values
## In the data frame, there are a lot of missing values that can be handled by filling those by specifying the value in the fill_value parameter.

table4 = pd.pivot_table(data=df, 
                        index='Platform', 
                        values=['Sales', 'Total_Year'],
                        columns=['Publishers'],
                        aggfunc={'Sales': np.sum, 'Total_Year': np.mean},
                        fill_value='MISSING')
table4



Sales                                           \
Publishers     Electronic Arts Nintendo PUBG Corporation Rockstar Games   
Platform                                                                  
Multi-platform           100.0  MISSING          MISSING          160.0   
PC                     MISSING  MISSING             75.0        MISSING   
Wii                    MISSING     82.0          MISSING        MISSING   

                                      Total_Year                            \
Publishers     Xbox Game Studios Electronic Arts Nintendo PUBG Corporation   
Platform                                                                     
Multi-platform             238.0            14.0  MISSING          MISSING   
PC                       MISSING         MISSING  MISSING              5.0   
Wii                      MISSING         MISSING     10.0          MISSING   

                                                 
Publishers     Rockstar Games Xbox Game Studios  
Platform                                         
Multi-platform            7.0               9.0  
PC                    MISSING           MISSING  
Wii                   MISSING           MISSING

In [ ]:
# Calculate row and column total
## Next, let’s examine the sales totals of each category of the platform. To do this, we will use the margins and margins_name parameters.

table5 = pd.pivot_table(data=df, 
                        index=['Platform'],
                        values='Sales',
                        aggfunc=['sum', 'mean', 'count'],
                        margins=True,
                        margins_name='Grand Total')
table5



In [ ]:
# We can observe in that output that a new index is added as a total which contains the total sales for different aggregation functions applied on the sales column.

# Multi-level index pivot table
## Based on the above pivot tables, only one feature was used in the index, i.e., a single level index.
# We can, however, create pivot tables using multiple indices. Whenever data is organized hierarchically,
# a pivot table with multi-level indexes can provide very useful and detailed summary information.

table6 = pd.pivot_table(data=df,
                        index=['Platform', 'Genre'],
                        values='Sales',
                        aggfunc=['sum', 'mean', 'count'],
                        margins=True,
                        margins_name='Grand Total')
table6



In [44]:
###############
###############
###############

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

X,y = fetch_openml("autos", version=1, as_frame=True, return_X_y=True)
data = X
data['target'] = y

pivot = np.round(pd.pivot_table(data, values='price', 
                                index='num-of-doors', 
                                columns='fuel-type', 
                                aggfunc=np.mean),2)
pivot

fuel-type,diesel,gas
num-of-doors,,
four,16432.38,13092.81
two,14350.00,12762.76


In [45]:
df

,Name,Genre,Platform,Publishers,Total_Year,Sales
0,Minecraft,"Survival,Sandbox",Multi-platform,Xbox Game Studios,9,238
1,Grand Theft Auto V,Action-adventure,Multi-platform,Rockstar Games,7,160
2,Tetris (EA),Puzzle,Multi-platform,Electronic Arts,14,100
3,Wii Sports,Sports simulation,Wii,Nintendo,10,82
4,PUBG: Battlegrounds,Battle royale,PC,PUBG Corporation,5,75
